<a href="https://colab.research.google.com/github/allan-gon/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [3]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


##Imports

In [4]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse
from sklearn.model_selection import train_test_split as tts

#Cleaning

#####Changing to datetime because easier to work with

In [5]:
df['created'] = pd.to_datetime(df['created'])

In [6]:
#make year and month feature
def make_month(date_time):
  return date_time.month

def make_year(date_time):
  return date_time.year

In [7]:
df['month'] = df['created'].apply(make_month)
df['year'] = df['created'].apply(make_year)
df['interest_level'].replace({'low':0,'medium':1,'high':2},inplace=True)

In [8]:
useable = df[((3 < df['month']) & (df['month'] < 7)) & (df['year'] == 2016)]

##Train Test Split

In [9]:
train = useable[useable['month'] != 6].copy()
test = useable[useable['month'] == 6].copy()

train.drop(['created','description','display_address','street_address',
            'month','year'],axis=1,inplace=True)
test.drop(['created','description','display_address','street_address',
            'month','year'],axis=1,inplace=True)

In [10]:
X_train = train.drop('price',axis=1)
y_train = train['price']

X_test = test.drop('price',axis=1)
y_test = test['price']

##Baseline

Will be predicting price by most features

In [11]:
y_pred = [y_train.mean()] * len(y_train)

mean_abs_er = mae(y_train, y_pred)
print(f'Train Error (April-May): {mean_abs_er:.2f} dollars')

Train Error (April-May): 1201.88 dollars


In [12]:
y_pred = [y_train.mean()] * len(y_test)
mean_abs_er = mae(y_test, y_pred)
print(f'Test Error (June): {mean_abs_er:.2f} dollars')

Test Error (June): 1197.71 dollars


#Linear Regression

#####Instantiate class

In [13]:
model = LR()

#####Train the model

In [14]:
model.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

######Use error metrics

######Mean Absolute Error

In [15]:
mean_abs_er = mae(y_train, model.predict(X_train))
print('Training error:', mean_abs_er)

Training error: 673.2867749623326


In [16]:
mean_abs_er = mae(y_test, model.predict(X_test))
print('Test error:', mean_abs_er)

Test error: 676.889002309455


######Mean Square Error

In [17]:
mean_sqr_er = mse(y_train,model.predict(X_train))
print('Training error:', mean_sqr_er)

Training error: 1128209.340477661


In [18]:
mean_sqr_er = mae(y_test, model.predict(X_test))
print('Test error:', mean_sqr_er)

Test error: 676.889002309455


#####Model coefficients and intercept

In [19]:
model.coef_

array([  1708.78941307,    489.9746021 ,   1238.99776005, -13150.46694504,
         -410.71620831,    146.30884652,    -32.5885334 ,   -147.60433018,
           75.13090821,    426.7249965 ,     53.45498711,   -103.38967576,
         -106.81896649,    114.31515752,    -91.31732603,    477.81293576,
         -166.70465428,    -81.35955479,    260.04141854,   -294.28930594,
          -43.70576234,     56.2571529 ,   -134.05555401,    180.09316258,
          146.38227363,    134.25464575,    -40.97308694,    181.7298944 ,
         -155.33757419])

In [20]:
model.intercept_

-1022604.9536483203

In [21]:
#X_train, X_test, y_train, y_test
X_train.head()

,bathrooms,bedrooms,latitude,longitude,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
2,1.0,1,40.7388,-74.0018,2,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,40.7539,-73.9677,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,40.8241,-73.9493,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.0,4,40.7429,-74.0028,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1.0,2,40.8012,-73.9660,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#####This removes longitude and latitude and reruns the model

In [22]:
lr = LR()
lr.fit(X_train.drop(['latitude','longitude'],axis=1), y_train)
mean_abs_er = mae(y_train, lr.predict(X_train.drop(['latitude','longitude'],axis=1)))
print('Training error:', mean_abs_er)

mean_abs_er = mae(y_test, lr.predict(X_test.drop(['latitude','longitude'],axis=1)))
print('Test error:', mean_abs_er)

Training error: 727.5019613417003
Test error: 728.0816740581698


#####Model 3 for fun, alot smaller

In [23]:
model3 = LR()
model3.fit(X_train[['bathrooms','bedrooms']],y_train)

mean_abs_er = mae(y_train, model3.predict(X_train[['bathrooms','bedrooms']]))
print('Training error:', mean_abs_er)

mean_abs_er = mae(y_test, model3.predict(X_test[['bedrooms','bathrooms']]))
print('Test error:', mean_abs_er)

Training error: 818.5310213271714
Test error: 1686.3988954577221


#Takeaway

In [24]:
print(f"If we guessed that apartment prices where always the mean of {y_train.mean()} we would be wrong of by $1197.71")
print("But if we used the regression model we would be off by $676.89")

If we guessed that apartment prices where always the mean of 3575.604007034292 we would be wrong of by $1197.71
But if we used the regression model we would be off by $676.89


#Feature Engineer

In [25]:
def cat_and_dog(col1, col2):
  x = col1 + col2
  temp = []
  for i in x:  
    if i == 2:
      temp.append(1)
  else:
    temp.append(0)
  return pd.Series(temp)

df['cats and dogs'] = cat_and_dog(df['cats_allowed'],df['dogs_allowed'])

In [26]:
def has_description(col):
  try:
    if col.strip() == "":
      return 0
    else:
      return 1
  except:
    return 0
    
df['has description'] = df['description'].apply(has_description)

In [27]:
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)
#df.drop('level_0',axis=1,inplace=True)

In [28]:
def len_description(col):
  temp =[]
  for i in range(len(col)):
    if col[i] == 0:
      temp.append(0)
    else:
      temp.append(len(df['description'][i]))
  return pd.Series(temp)

df['description length'] = len_description(df['has description'])

In [29]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space,month,year,cats and dogs,has description,description length
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,2016,1.0,1,588
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,2016,1.0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,2,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2016,1.0,1,691
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2016,1.0,1,492
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,2016,1.0,1,479
